In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F
import numpy as np
import pandas as pd

In [25]:
model_name = "prajjwal1/bert-tiny"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, output_hidden_states=True)

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
device = torch.device('cpu')

In [4]:
from tqdm import tqdm

In [6]:
eng_desc_df = pd.read_csv('../data/games_with_eng_descriptions_clear.csv')
#eng_desc_df = eng_desc_df[eng_desc_df['short_description'].notna()]
#ru_desc_df = ru_desc_df.reset_index(drop=True)
desc_new = eng_desc_df['description']
eng_desc_df

,Unnamed: 0,Name,Date,Developer,Publisher,Genre,Price,Langs,DLC,Mature,Single,Total Review Cnt,Pos Review Cnt,Neg Review Cnt,Score,appid,Tags,description
0,0,Senren＊Banka,"14 Feb, 2020",YUZUSOFT,HIKARI FIELD,"['Adventure', 'Casual']","29,99€","['English', 'Japanese', 'Simplified Chinese', ...",0.0,1.0,1.0,14158.0,14033.0,125.0,0.963524,1144400,"['Visual Novel', 'Casual', 'Adventure', 'Datin...",Senren＊Banka is a Japanese-style visual novel ...
1,1,A Short Hike,"30 Jul, 2019",adamgryu,adamgryu,"['Adventure', 'Indie']","6,59€","['English', 'French', 'Spanish - Latin America...",0.0,0.0,1.0,13541.0,13442.0,99.0,0.964582,1055540,"['Adventure', 'Exploration', 'Indie', 'Cute', ...","Hike, climb, and soar through the peaceful mou..."
2,2,Papa's Freezeria Deluxe,"31 Mar, 2023",Flipline Studios,Flipline Studios,"['Action', 'Casual', 'Indie', 'Simulation', 'S...","5,89€",['English'],0.0,0.0,1.0,3898.0,3864.0,34.0,0.950507,2291760,"['Cooking', 'Strategy', 'Time Management', 'Si...",Return to Calypso Island to serve seasonal sun...
3,3,TOEM,"17 Sep, 2021",Something We Made,Something We Made,"['Adventure', 'Casual', 'Indie']","19,50€","['English', 'French', 'German', 'Spanish - Spa...",1.0,0.0,1.0,3736.0,3706.0,30.0,0.950617,1307580,"['Cozy', 'Wholesome', 'Adventure', 'Puzzle', '...",Set off on a delightful expedition and use you...
4,4,Ib,"11 Apr, 2022",kouri,PLAYISM,"['Adventure', 'Indie']","10,79€","['English', 'Japanese', 'Simplified Chinese', ...",0.0,0.0,1.0,2617.0,2596.0,21.0,0.945946,1901370,"['Horror', '2D', 'RPGMaker', 'Psychological Ho...",A 2D exploration adventure game set in a creep...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4518,4518,Borderlands 3: Retro Cosmetic Pack,"13 Mar, 2020",Gearbox Software,2K,"['Action', 'RPG']",499.0,"['English', 'French', 'Italian', 'German', 'Sp...",0.0,1.0,1.0,NaN,NaN,NaN,0.415901,1232254,"['Action', 'RPG', 'Gore', 'Violent']",The Retro Cosmetic Pack includes a Retro Vault...
4519,4519,Tomb Raider: Tomb of the Lost Adventurer,"26 Mar, 2013",Crystal Dynamics,Crystal Dynamics,"['Action', 'Adventure']",299.0,"['English', 'German', 'French', 'Italian', 'Ko...",0.0,0.0,1.0,NaN,NaN,NaN,0.410642,208812,"['Action', 'Adventure']",Explore the final resting place of an intrepid...
4520,4520,Sid Meier's Civilization® VI: Persia and Maced...,"28 Mar, 2017",Firaxis Games,2K,['Strategy'],20985.0,"['English', 'French', 'Italian', 'German', 'Sp...",0.0,0.0,1.0,NaN,NaN,NaN,0.401544,512035,['Strategy'],Alexander leads Macedon and Cyrus leads Persia...
4521,4521,Crusader Kings III: Friends & Foes,"8 Sep, 2022",Paradox Development Studio,Paradox Interactive,"['RPG', 'Simulation', 'Strategy']",499.0,"['English', 'French', 'German', 'Spanish - Spa...",0.0,0.0,1.0,NaN,NaN,NaN,0.375143,2114760,"['Strategy', 'RPG', 'Simulation', 'Psychologic...","Beware, ruler - tread carefully with those you..."


In [8]:
embeddings_eng_desc = []

embeddings = [embeddings_eng_desc] 
for i in range(0,len(embeddings)):
    
    if i == 0:
        length = 32
    embedding =embeddings[i]
    for description in tqdm(desc_new):
        description_tokens = tokenizer.encode_plus(
            description,
            add_special_tokens=True,
            max_length=length, # Ограничение на максимальную длину входной последовательности
            pad_to_max_length=True, # Дополним последовательность нулями до максимальной длины если слов меньше заданной длины
            return_tensors='pt' # Вернём тензоры PyTorch
        )

        with torch.no_grad():
            # Получим эмбеддинг аннотации
            outputs = model(**description_tokens)
            hidden_states = outputs.hidden_states
            last_hidden_state = hidden_states[-1][:,0,:] # Используем предпоследний слой для эмбеддинга
            last_hidden_state = torch.nn.functional.normalize(last_hidden_state)

            embedding.append((last_hidden_state).squeeze())

100%|██████████████████████████████████████| 4523/4523 [00:14<00:00, 316.18it/s]


In [26]:

queries = [
"Puzzle with complicated levels",
"Amazing Open World RPG",
"Strategy with well-built economic system",
"Breathtaking novel with nice story",
"First person shooter with up-to-date physics",
"Relaxing indie with cute graphics"
          ]
#embeddings = [embeddings64x, embeddings_64xxx, embeddings_128xxx, embeddings_256xxx, embeddings_256, embeddings_512]          
#embeddings = [embeddings_ru_desc]
for query in queries:
    print(f'\n{query}')
    length = 32

    query_tokens = tokenizer.encode_plus(
        query,
        add_special_tokens=True,
        max_length=length, # Здесь вставь правильную длину !!!!!!!!!!!!!!!! 
        pad_to_max_length=True, # Дополним последовательность нулями до максимальной длины
        return_tensors='pt' # Вернём тензоры PyTorch
    )
    with torch.no_grad():
        query_outputs = model(**query_tokens)
        query_hidden_states = query_outputs.hidden_states
        query_last_hidden_state = query_hidden_states[-1][:,0,:] # Используем предпоследний слой для эмбеддинга
        query_embedding = query_last_hidden_state.squeeze()
    cosine_similarities = torch.nn.functional.cosine_similarity(
    query_embedding.unsqueeze(0),
    torch.stack(embeddings_eng_desc) # И здесь вставь правильный эмбединг!!!!!!!
    )

    cosine_similarities = cosine_similarities.numpy()

    indices = np.argsort(cosine_similarities)[::-1] # Сортировка по убыванию
    #rec_titles = []

    for i in indices[:5]:
        print(eng_desc_df['Name'][i], f'https://store.steampowered.com/app/{eng_desc_df["appid"][i]}', eng_desc_df['description'][i]) 

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Puzzle with complicated levels
Zup! S https://store.steampowered.com/app/617670 Minimal physical puzzle with explosions
Zup! F https://store.steampowered.com/app/601220 Minimal physical puzzle with explosions
Zup! Q https://store.steampowered.com/app/2206560 Minimal physical puzzle with explosions
The Entropy Centre https://store.steampowered.com/app/1730590 Think in reverse. Outsmart the impossible. A mind-bending first-person adventure where you solve ingenious and complex puzzles by rewinding objects through time.
Turing Complete https://store.steampowered.com/app/1444480 Learn CPU architecture with puzzles

Amazing Open World RPG
Tabletop Creator https://store.steampowered.com/app/861590 The ultimate way to design Tabletop Games.
UnReal World https://store.steampowered.com/app/351700 The cult classic open-ended wilderness survival roguelike RPG taking place in the ancient North. Throughout the years, UnReal World has been praised for its incredible depth, realism, atmosphere and i

In [10]:
embeddings_eng_desc64 = []
embeddings_eng_desc128 = []
embeddings_eng_desc256 = []

embeddings = [embeddings_eng_desc64, embeddings_eng_desc128, embeddings_eng_desc256] 
for i in range(0,len(embeddings)):
    
    if i == 0:
        length = 64
    elif i ==1:
        length = 128
    elif i == 2:
        length = 256
    embedding =embeddings[i]
    for short_description in tqdm(desc_new):
        short_description_tokens = tokenizer.encode_plus(
            short_description,
            add_special_tokens=True,
            max_length=length, # Ограничение на максимальную длину входной последовательности
            pad_to_max_length=True, # Дополним последовательность нулями до максимальной длины если слов меньше заданной длины
            return_tensors='pt' # Вернём тензоры PyTorch
        )

        with torch.no_grad():
            # Получим эмбеддинг аннотации
            outputs = model(**short_description_tokens)
            hidden_states = outputs.hidden_states
            last_hidden_state = hidden_states[-1][:,0,:] # Используем предпоследний слой для эмбеддинга
            last_hidden_state = torch.nn.functional.normalize(last_hidden_state)

            embedding.append((last_hidden_state).squeeze())

100%|██████████████████████████████████████| 4523/4523 [00:27<00:00, 163.61it/s]


In [27]:

queries = [
"Puzzle with complicated levels",
"Amazing Open World RPG",
"Strategy with well-built economic system",
"Breathtaking novel with nice story",
"First person shooter with up-to-date physics",
"Relaxing indie with cute graphics"
          ]
#embeddings = [embeddings64x, embeddings_64xxx, embeddings_128xxx, embeddings_256xxx, embeddings_256, embeddings_512]          
embeddings = [embeddings_eng_desc64, embeddings_eng_desc128, embeddings_eng_desc256] 
for query in queries:
    print(f'\n{query}')
    for i in range(0,len(embeddings)): # Если используешь 1 эмбединг, то заккоментируй отсюда
        
        if i == 0:
            length = 64
        elif i == 1:
            length = 128
        elif i == 2:
            length = 256
        # elif i == 3:
        #     length = 256
        # elif i == 4:
        #     length = 256
        # else:
        #     length = 512
        print(f'\n{length}')
        
        embedding =embeddings[i] # ДО СЮДА
    
        
        query_tokens = tokenizer.encode_plus(
            query,
            add_special_tokens=True,
            max_length=length, # Здесь вставь правильную длину !!!!!!!!!!!!!!!! 
            pad_to_max_length=True, # Дополним последовательность нулями до максимальной длины
            return_tensors='pt' # Вернём тензоры PyTorch
        )
        with torch.no_grad():
            query_outputs = model(**query_tokens)
            query_hidden_states = query_outputs.hidden_states
            query_last_hidden_state = query_hidden_states[-1][:,0,:] # Используем предпоследний слой для эмбеддинга
            query_embedding = query_last_hidden_state.squeeze()
        cosine_similarities = torch.nn.functional.cosine_similarity(
        query_embedding.unsqueeze(0),
        torch.stack(embedding) # И здесь вставь правильный эмбединг!!!!!!!
        )

        cosine_similarities = cosine_similarities.numpy()

        indices = np.argsort(cosine_similarities)[::-1] # Сортировка по убыванию
        #rec_titles = []
        
        for i in indices[:5]:
            print(eng_desc_df['Name'][i], f'https://store.steampowered.com/app/{eng_desc_df["appid"][i]}', eng_desc_df['description'][i]) 


Puzzle with complicated levels

64
Zup! S https://store.steampowered.com/app/617670 Minimal physical puzzle with explosions
Zup! Q https://store.steampowered.com/app/2206560 Minimal physical puzzle with explosions
Zup! F https://store.steampowered.com/app/601220 Minimal physical puzzle with explosions
Turing Complete https://store.steampowered.com/app/1444480 Learn CPU architecture with puzzles
The Entropy Centre https://store.steampowered.com/app/1730590 Think in reverse. Outsmart the impossible. A mind-bending first-person adventure where you solve ingenious and complex puzzles by rewinding objects through time.

128
Zup! S https://store.steampowered.com/app/617670 Minimal physical puzzle with explosions
Zup! F https://store.steampowered.com/app/601220 Minimal physical puzzle with explosions
Zup! Q https://store.steampowered.com/app/2206560 Minimal physical puzzle with explosions
Turing Complete https://store.steampowered.com/app/1444480 Learn CPU architecture with puzzles
The Entro

In [13]:
model_name = "prajjwal1/bert-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, output_hidden_states=True)

Some weights of the model checkpoint at prajjwal1/bert-medium were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
embeddings_eng_desc64_medium = []
#embeddings_eng_desc128_medium = []
#embeddings_eng_desc256_medium = []

embeddings = [embeddings_eng_desc64_medium]#, embeddings_eng_desc128_medium, embeddings_eng_desc256_medium] 
for i in range(0,len(embeddings)):
    
    if i == 0:
        length = 64
    elif i ==1:
        length = 128
    elif i == 2:
        length = 256
    embedding =embeddings[i]
    for short_description in tqdm(desc_new):
        short_description_tokens = tokenizer.encode_plus(
            short_description,
            add_special_tokens=True,
            max_length=length, # Ограничение на максимальную длину входной последовательности
            pad_to_max_length=True, # Дополним последовательность нулями до максимальной длины если слов меньше заданной длины
            return_tensors='pt' # Вернём тензоры PyTorch
        )

        with torch.no_grad():
            # Получим эмбеддинг аннотации
            outputs = model(**short_description_tokens)
            hidden_states = outputs.hidden_states
            last_hidden_state = hidden_states[-1][:,0,:] # Используем предпоследний слой для эмбеддинга
            last_hidden_state = torch.nn.functional.normalize(last_hidden_state)

            embedding.append((last_hidden_state).squeeze())

100%|███████████████████████████████████████| 4523/4523 [02:49<00:00, 26.70it/s]


In [19]:
len(embeddings_eng_desc64_medium[0])

512

In [17]:

queries = [
"Puzzle with complicated levels",
"Amazing Open World RPG",
"Strategy with well-built economic system",
"Breathtaking novel with nice story",
"First person shooter with up-to-date physics",
"Relaxing indie with cute graphics"
          ]
#embeddings = [embeddings64x, embeddings_64xxx, embeddings_128xxx, embeddings_256xxx, embeddings_256, embeddings_512]          
embeddings = [embeddings_eng_desc64_medium]#, embeddings_eng_desc128, embeddings_eng_desc256] 
for query in queries:
    print(f'\n{query}')
    for i in range(0,len(embeddings)): # Если используешь 1 эмбединг, то заккоментируй отсюда
        
        if i == 0:
            length = 64
        elif i == 1:
            length = 64
        elif i == 2:
            length = 256
        # elif i == 3:
        #     length = 256
        # elif i == 4:
        #     length = 256
        # else:
        #     length = 512
        
        
        embedding =embeddings[i] # ДО СЮДА
        print(f'\n{length}{embedding}')
        
        query_tokens = tokenizer.encode_plus(
            query,
            add_special_tokens=True,
            max_length=length, # Здесь вставь правильную длину !!!!!!!!!!!!!!!! 
            pad_to_max_length=True, # Дополним последовательность нулями до максимальной длины
            return_tensors='pt' # Вернём тензоры PyTorch
        )
        with torch.no_grad():
            query_outputs = model(**query_tokens)
            query_hidden_states = query_outputs.hidden_states
            query_last_hidden_state = query_hidden_states[-1][:,0,:] # Используем предпоследний слой для эмбеддинга
            query_embedding = query_last_hidden_state.squeeze()
        cosine_similarities = torch.nn.functional.cosine_similarity(
        query_embedding.unsqueeze(0),
        torch.stack(embedding) # И здесь вставь правильный эмбединг!!!!!!!
        )

        cosine_similarities = cosine_similarities.numpy()

        indices = np.argsort(cosine_similarities)[::-1] # Сортировка по убыванию
        #rec_titles = []
        
        for i in indices[:5]:
            print(eng_desc_df['Name'][i], f'https://store.steampowered.com/app/{eng_desc_df["appid"][i]}', eng_desc_df['description'][i]) 


Puzzle with complicated levels


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [28]:
model_name = "prajjwal1/bert-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name, output_hidden_states=True)

Some weights of the model checkpoint at prajjwal1/bert-small were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
embeddings_eng_desc64_small = []
#embeddings_eng_desc128_medium = []
#embeddings_eng_desc256_medium = []

embeddings = [embeddings_eng_desc64_small]#, embeddings_eng_desc128_medium, embeddings_eng_desc256_medium] 
for i in range(0,len(embeddings)):
    
    if i == 0:
        length = 64
    elif i ==1:
        length = 128
    elif i == 2:
        length = 256
    embedding =embeddings[i]
    for short_description in tqdm(desc_new):
        short_description_tokens = tokenizer.encode_plus(
            short_description,
            add_special_tokens=True,
            max_length=length, # Ограничение на максимальную длину входной последовательности
            pad_to_max_length=True, # Дополним последовательность нулями до максимальной длины если слов меньше заданной длины
            return_tensors='pt' # Вернём тензоры PyTorch
        )

        with torch.no_grad():
            # Получим эмбеддинг аннотации
            outputs = model(**short_description_tokens)
            hidden_states = outputs.hidden_states
            last_hidden_state = hidden_states[-1][:,0,:] # Используем предпоследний слой для эмбеддинга
            last_hidden_state = torch.nn.functional.normalize(last_hidden_state)

            embedding.append((last_hidden_state).squeeze())

100%|███████████████████████████████████████| 4523/4523 [01:29<00:00, 50.64it/s]


In [29]:

queries = [
"Puzzle with complicated levels",
"Amazing Open World RPG",
"Strategy with well-built economic system",
"Breathtaking novel with nice story",
"First person shooter with up-to-date physics",
"Relaxing indie with cute graphics"
          ]
#embeddings = [embeddings64x, embeddings_64xxx, embeddings_128xxx, embeddings_256xxx, embeddings_256, embeddings_512]          
embeddings = [embeddings_eng_desc64_small]#, embeddings_eng_desc128, embeddings_eng_desc256] 
for query in queries:
    print(f'\n{query}')
    for i in range(0,len(embeddings)): # Если используешь 1 эмбединг, то заккоментируй отсюда
        
        if i == 0:
            length = 64
        elif i == 1:
            length = 64
        elif i == 2:
            length = 256
        # elif i == 3:
        #     length = 256
        # elif i == 4:
        #     length = 256
        # else:
        #     length = 512
        
        
        embedding =embeddings[i] # ДО СЮДА
        print(f'\n{length}{embedding}')
        
        query_tokens = tokenizer.encode_plus(
            query,
            add_special_tokens=True,
            max_length=length, # Здесь вставь правильную длину !!!!!!!!!!!!!!!! 
            pad_to_max_length=True, # Дополним последовательность нулями до максимальной длины
            return_tensors='pt' # Вернём тензоры PyTorch
        )
        with torch.no_grad():
            query_outputs = model(**query_tokens)
            query_hidden_states = query_outputs.hidden_states
            query_last_hidden_state = query_hidden_states[-1][:,0,:] # Используем предпоследний слой для эмбеддинга
            query_embedding = query_last_hidden_state.squeeze()
        cosine_similarities = torch.nn.functional.cosine_similarity(
        query_embedding.unsqueeze(0),
        torch.stack(embedding) # И здесь вставь правильный эмбединг!!!!!!!
        )

        cosine_similarities = cosine_similarities.numpy()

        indices = np.argsort(cosine_similarities)[::-1] # Сортировка по убыванию
        #rec_titles = []
        
        for i in indices[:5]:
            print(eng_desc_df['Name'][i], f'https://store.steampowered.com/app/{eng_desc_df["appid"][i]}', eng_desc_df['description'][i]) 


Puzzle with complicated levels


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



KeyboardInterrupt: 

In [23]:
!jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

  _   _          _      _
 | | | |_ __  __| |__ _| |_ ___
 | |_| | '_ \/ _` / _` |  _/ -_)
  \___/| .__/\__,_\__,_|\__\___|
       |_|
                       
Read the migration plan to Notebook 7 to learn about the new features and the actions to take if you are using extensions.

https://jupyter-notebook.readthedocs.io/en/latest/migrate_to_notebook7.html

Please note that updating to Notebook 7 might break some of your extensions.

[W 15:58:02.081 NotebookApp] Loading JupyterLab as a classic notebook (v6) extension.
[C 15:58:02.081 NotebookApp] You must use Jupyter Server v1 to load JupyterLab as notebook extension. You have v2.5.0 installed.
    You can fix this b